In [3]:
import numpy as np
import numpy.random as random
import os
import librosa.feature
from tqdm import tqdm
from tensorflow.keras.utils import Sequence
import math

class GenData(Sequence):
    def extract_features(self, file):
        wav = librosa.load(file, sr=16E3)
        mfcc = librosa.feature.mfcc(*wav, n_mfcc=self.num_mfcc, win_length=320, hop_length=320, n_fft=512, window='hann')
        return mfcc

    def __init__(self, path, digits, split, batch_size):
        self.batch_size = batch_size
        self.digits = digits
        self.num_classes = len(digits)
        self.num_mfcc = 30
        self.y = []
        self.X = []

        test_list = open(os.path.join(path, 'testing_list.txt')).read().split('\n')
        val_list = open(os.path.join(path, 'validation_list.txt')).read().split('\n')

        for idx_digit, str_digit in enumerate(digits):
            path_digit = os.path.join(path, str_digit)
            files = os.listdir(path_digit)
            for f in tqdm(files):
                f = str_digit + '/' + f
                if split=='test' and f not in test_list:
                    continue
                elif split=='val' and f not in val_list:
                    continue
                elif split=='train' and (f in test_list or f in val_list):
                    continue

                f = os.path.join(path, f)
                self.X.append(self.extract_features(f))
                self.y.append(np.zeros((self.num_classes,)))
                self.y[-1][idx_digit] = 1

        self.idx = random.permutation(len(self.y))

    def __len__(self):
        return math.floor(len(self.X) / self.batch_size)

    def __getitem__(self, item):
        idx_batch = self.idx[item*self.batch_size : (item+1)*self.batch_size]

        maxlen = max([self.X[it].shape[1] for it in idx_batch])
        X_batch = np.zeros((self.batch_size, maxlen, self.num_mfcc))
        y_batch = np.zeros((self.batch_size, self.num_classes))

        for it, rand_it in enumerate(idx_batch):
            sequence = self.X[rand_it]
            X_batch[it, -sequence.shape[1]:, :] = sequence.T
            y_batch[it, :] = self.y[rand_it]

        return X_batch, y_batch

    def on_epoch_end(self):
        self.idx = random.permutation(len(self.X))

In [4]:
batch_size = 32
digits = ['five', 'six', 'seven', 'eight']
path = 'D:\\RWTH Aachen\\Audio Processing Using Python\\Lab Assignment 07\\Training Data Lab Course 7\\'
train_data = GenData(path, split='train', batch_size=batch_size,  digits=digits)
val_data = GenData(path, split='val', batch_size=batch_size,  digits=digits)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\RWTH Aachen\\Audio Processing Using Python\\Lab Assignment 07\\Training Data Lab Course 7\\testing_list.txt'

In [ ]:
x0, y0 = train_data.__getitem__(0)
print(x0.shape)
print(y0.shape)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, InputLayer

model = Sequential()
model.add(InputLayer((None, train_data.num_mfcc)))
model.add(BatchNormalization())
model.add(Dense(32, activation='linear'))
model.add(LSTM(32,activation='tanh', dropout=0.05,  return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(32,activation='tanh', dropout=0.05,  return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(32,activation='tanh', dropout=0.05, return_sequences=False))

model.add(Dense(train_data.num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
opt = Adam(learning_rate=0.001)
loss = CategoricalCrossentropy()
acc = CategoricalAccuracy()
model.compile(optimizer=opt, loss=loss, metrics=[acc])

In [ ]:
import matplotlib.pyplot as plt
history =  model.fit(train_data, validation_data=val_data, epochs=10)
%matplotlib qt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.xlabel('Epochs')
plt.legend()

In [ ]:
test_data = GenData(path, split='test', batch_size=1, digits=digits)
acc_test = model.evaluate(test_data)
print(acc_test)

In [ ]:
from tensorflow.keras.layers import Add
from tensorflow.keras import Model, Input

input = Input((None, train_data.num_mfcc))
norm = BatchNormalization()(input)
x1 = Dense(32, activation='linear')(norm)

x = LSTM(32, dropout=0.05, return_sequences=True)(x1)
x = BatchNormalization()(x)
delta = LSTM(32, dropout=0.05, return_sequences=True)(x)

s = Add()([delta, x1])
x = BatchNormalization()(s)
x = LSTM(32, dropout=0.05, return_sequences=False)(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

resnet = Model(inputs=input, outputs=output)
resnet.compile(optimizer=opt, loss=loss, metrics=[acc])

In [ ]:
resnet.fit(train_data, epochs=10, validation_data=val_data)
resnet.evaluate(test_data)

In [ ]:
submodel = Model(inputs=input, outputs=[delta, s])
submodel.compile(optimizer=opt, loss=loss, metrics=[acc])
x0, y0 = test_data.__getitem__(0)
delta_out, s_out = submodel.predict(x0)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(delta_out[0])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from shutil import rmtree
logdir = 'logs'
if os.path.isdir(logdir):
    rmtree(logdir)
os.mkdir(logdir)
tensorboard = TensorBoard(log_dir=logdir, histogram_freq=0, profile_batch=0)

# resnet.fit(train_data, validation_data=val_data, epochs=100, callbacks=[early_stopping, tensorboard])

In [ ]:
print(os.path.isdir(logdir))
print(logdir)
rmtree(logdir)
print(os.path.isdir(logdir))

In [ ]:
from kerastuner import HyperParameters

def model_builder(hp: HyperParameters):
    num_blocks = hp.Int('blocks', min_value=1, max_value=3)
    num_units = hp.Choice('units', values=[16, 32, 48])
    dropout = hp.Float('dropout', min_value=0, max_value=0.3, step=0.1)
    with_skips = hp.Boolean('skips')

    input = Input((None, train_data.num_mfcc))
    norm = BatchNormalization()(input)
    x1 = Dense(num_units, activation='linear')(norm)

    for _ in range(num_blocks):
        x = LSTM(num_units, dropout=dropout, return_sequences=True)(x1)
        x = BatchNormalization()(x)
        delta = LSTM(num_units, dropout=dropout, return_sequences=True)(x)
        if with_skips:
            s = Add()([delta, x1])
        else:
            s = delta
        x1 = BatchNormalization()(s)

    x = LSTM(num_units, dropout=dropout, return_sequences=False)(x1)
    output = Dense(train_data.num_classes, activation='softmax')(x)

    model = Model(inputs=input, outputs=output)
    model.compile(optimizer=opt, loss=loss, metrics=[acc])
    return model

In [ ]:
train_data.batch_size = 32
val_data.batch_size = 32
from kerastuner.tuners import Hyperband

tuner = Hyperband(model_builder,
                  objective='val_categorical_accuracy',
                  factor=4,
                  directory='tuner_checkpoints',
                  project_name='hello_tuner',
                  max_epochs=20,
                  overwrite=True
                  )
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5),
    TensorBoard(log_dir=logdir, histogram_freq=0, profile_batch=0)
]
tuner.search(train_data, validation_data=val_data, callbacks=callbacks)


In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
model.fit(train_data, validation_data=val_data, epochs=50, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])
model.evaluate(test_data)
